In [1]:
from __future__ import print_function, division
import os, sys
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

In [2]:
dataroot = '/home/abhijit/Downloads/video frames/crowd/'
csv = '/home/abhijit/Downloads/video frames/optical flow/optical flow/crowdgroundtruth.xlsx'
dataroot_2 = '/home/abhijit/Downloads/video frames/optical flow/'

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


# Reading The Ground Truth


In [ ]:
all_ = pd.read_excel(csv, header = 0)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(all_, test_size=0.3, random_state = 0)
valid_df, test_df = train_test_split(test_df, test_size=.66, random_state = 0)

# DataLoader for Cycle GAN


In [ ]:
class GANDataset(Dataset):
    

    def __init__(self, csv_file,root_dir,df,root_dir_2, transform=None):

        self.landmarks_frame = pd.read_excel(csv_file, header = 0)
        self.read_number = pd.read_excel(csv_file,header = 1)
        self.root_dir = root_dir
        self.root_dir_2 = root_dir_2
        self.transform = transform
        self.df = df

    def __len__(self):
        #return len(os.listdir(self.root_dir_2))
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.df.iloc[idx]['Frame1']
        number_left = self.df.iloc[idx,3]
        number_right = self.df.iloc[idx,4]
        #number_combined = self.df.iloc[idx,5]
        number_combined = number_left + number_right
        #number_all = self.landmarks_frame['left_to_right'][idx:idx]
        #number_all = 0
        number_all = np.array([number_left,number_right, number_combined])
        
        ix = ['scene1','scene2','scene3','scene4','scene5','scene6','scene7','scene8','scene9', 'scene10','scene11','scene12','scene13','scene14','scene15']
        jx = ['scene10','scene11','scene12','scene13','scene14','scene15']

#img_name = 'scene6medcam1frame780'
        i = ''
        x = 0
        for j in jx:
            if j in img_name:
                x = 1
                break
        i = j
        if x==0:
            for i in ix:
                if i in img_name:
                    break
        
        dense = ['low', 'med', 'high']
        cam = ['cam1', 'cam2', 'cam3']
        occlu = ['', 'occlu']   

        for den in dense:
            if den in img_name:
                break
        for c in cam:
            if c in img_name:
                break
            
        if 'occlu' in img_name:
            occ = 'occlu'
        else:
            occ= ''
        
        #image = io.imread(self.root_dir+'scene' + str(i)+'/'+'scene'+str(i)+den+c+occ+'/'+img_name )
        img = self.root_dir+ str(i)+'/'+str(i)+den+c+occ+'/'+img_name+'.jpg'
        
        #optical_flow = self.root_dir_2 + str(i)+'/'+str(i)+den+c+occ+'/'+img_name+'.png'
        image = io.imread(img)
        image = Image.fromarray(image)
        #op_flow = io.imread(optical_flow)
        #op_flow = Image.fromarray(op_flow)
        origi = image
        image = self.transform(image)
        #op_flow = self.transform(op_flow)
        
        
        #dirs2 = os.listdir(self.root_dir_2)
        
        #image_2 = self.root_dir_2+ '/' +dirs2[idx]   
        
        image_2 = img
        image_2 = Image.open(image_2)
        #image_2 = convert_gray2rgb(image_2)
        image_2 = image_2.convert("RGB")
        #image = convert_gray2rgb(image)
        
        
        image_2 = self.transform(image_2)
        
        every_thing = {'image': image,
                        'img_root':img,
                       'image_2': image_2,
                       'img_2_name': img_name,
                      'left_to_right': number_left,
                      'right_to_left': number_right,
                      'combined':number_combined,
                      'all':number_all
                }
        

        return every_thing

In [ ]:
from torchvision import transforms
train_dataset = GANDataset(csv, dataroot, all_,dataroot,
                                    transform = transforms.Compose([ transforms.Resize([256,256]),  transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])]))



In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=1, shuffle=True)

# Modified Cycle GAN Structure

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [  nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features),
                        nn.ReLU(inplace=True),
                        nn.ReflectionPad2d(1),
                        nn.Conv2d(in_features, in_features, 3),
                        nn.InstanceNorm2d(in_features)  ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9):
        super(Generator, self).__init__()

        # Initial convolution block       
        model = [   nn.ReflectionPad2d(3),
                    nn.Conv2d(input_nc, 64, 7),
                    nn.InstanceNorm2d(64),
                    nn.ReLU(inplace=True) ]

        # Downsampling
        in_features = 64
        out_features = in_features*2
        for _ in range(2):
            model += [  nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features*2

        # Residual blocks
        for _ in range(n_residual_blocks):
            model += [ResidualBlock(in_features)]

        # Upsampling
        out_features = in_features//2
        for _ in range(2):
            model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
                        nn.InstanceNorm2d(out_features),
                        nn.ReLU(inplace=True) ]
            in_features = out_features
            out_features = in_features//2

        # Output layer
        model += [  nn.ReflectionPad2d(3),
                    nn.Conv2d(64, output_nc, 7),
                    nn.Tanh() ]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        # A bunch of convolutions one after another
        model = [   nn.Conv2d(input_nc, 64, 4, stride=2, padding=1),
                    nn.LeakyReLU(0.2, inplace=True) ]

        #model += [  nn.Conv2d(32, 32, 4, stride=2, padding=1),
        #            nn.InstanceNorm2d(128), 
         #           nn.LeakyReLU(0.2, inplace=True) ]

        #model += [  nn.Conv2d(128, 256, 4, stride=2, padding=1),
         #           nn.InstanceNorm2d(256), 
          #          nn.LeakyReLU(0.2, inplace=True) ]

        #model += [  nn.Conv2d(64, 128, 4, padding=1),
            #        nn.InstanceNorm2d(512), 
          #          nn.LeakyReLU(0.2, inplace=True) ]

        # FCN classification layer
        model += [nn.Conv2d(64, 1, 4, padding=1)]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        x =  self.model(x)
        # Average pooling and flatten
        return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)

In [ ]:
netG_A2B = Generator(3, 3)
netG_B2A = Generator(3, 3)
netD_A = Discriminator(3)
netD_B = Discriminator(3)

In [ ]:
netG_A2B.cuda()
#torch.cuda.empty_cache()
netG_B2A.cuda()
#torch.cuda.empty_cache()
netD_A.cuda()
#torch.cuda.empty_cache()
netD_B.cuda()
#torch.cuda.empty_cache()

# Generator and Discriminator Initialization

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant(m.bias.data, 0.0)

In [ ]:
netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# SSIM 

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from math import exp

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window

def _ssim(img1, img2, window, window_size, channel, size_average = True):
    mu1 = F.conv2d(img1, window, padding = window_size//2, groups = channel)
    mu2 = F.conv2d(img2, window, padding = window_size//2, groups = channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding = window_size//2, groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = window_size//2, groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = window_size//2, groups = channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

class SSIM(torch.nn.Module):
    def __init__(self, window_size = 11, size_average = True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)
            
            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)
            
            self.window = window
            self.channel = channel


        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)

def ssim(img1, img2, window_size = 11, size_average = True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    
    return _ssim(img1, img2, window, window_size, channel, size_average)

# Losses for Cycle GAN

In [ ]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

# Optimizer for Cycle GAN Training

In [ ]:
import itertools

optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr= .0002, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr= .0002, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr= .0002, betas=(0.5, 0.999))

# Learning Rate Schedular for Cycle GAN training

In [ ]:
from torch.optim import lr_scheduler

class LambdaLR():
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert ((n_epochs - decay_start_epoch) > 0), "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch)/(self.n_epochs - self.decay_start_epoch)


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(50, 0, 25).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda = LambdaLR(50, 0 ,25).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(50, 0, 25).step)

# Replay Buffer to Hold the Previous outputs from Generator During Training

In [ ]:
from torch.autograd import Variable
import random

class ReplayBuffer():
    def __init__(self, max_size=50):
        assert (max_size > 0), 'Empty buffer or trying to create a black hole. Be careful.'
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0,1) > 0.5:
                    i = random.randint(0, self.max_size-1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [ ]:
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Training of Cycle GAN

In [ ]:
epochs = 50

for epoch in range(epochs):
    
    for i, data in enumerate(train_loader):        
        
        real_A = Variable(data['image'].cuda())
        real_B = Variable(data['image_2'].cuda())
        
        
        
        
        target_real = Variable(torch.tensor(len(data['image'])).fill_(1.0).float(), requires_grad=False).cuda()
        target_fake = Variable(torch.tensor(len(data['image'])).fill_(0.0).float(), requires_grad=False).cuda()
        
        
        #real_A = Variable(data['image'])
        #real_B = Variable(data['image_2'])        
        
        
        
        #target_real = Variable(torch.tensor(len(data['image'])).fill_(1.0).float(), requires_grad=False)
        #target_fake = Variable(torch.tensor(len(data['image'])).fill_(0.0).float(), requires_grad=False)
        
        
        
        
        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_A2B(B) should equal B if real B is fed
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B)* 5.0
        

        
        
        # G_B2A(A) should equal A if real A is fed
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A)* 5.0
        
        
        
        
        
        

        # GAN loss
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = (criterion_cycle(recovered_A, real_A)* 10.0)

         
        
        #seamese loss for 1st image
        #siamese_1.load_state_dict(torch.load('/home/abhijit/Documents/saved_models/Siamese_game.pth'))
        with torch.no_grad():       
         #   x1, x2 = siamese_1(real_A,fake_B)
            SSIM_B2A = 1 -ssim_loss(real_A,recovered_A).detach()
        
        
        #label = 0
 
        
            
        #if SSIM_B2A > .1:
         #   label = 0
        #else:
         #   label = 1 
        
        #same_loss_1 = 2* loss(x1, x2,1 )

        #p = same_loss_1
        
        

        
        
        
        #simease end for 1st image

        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = (criterion_cycle(recovered_B, real_B)* 10.0)
        
        
        
        
        
        #siamese_2.load_state_dict(torch.load('/home/abhijit/Documents/saved_models/Siamese_real.pth')) 
        
        #2nd siamese loss
        with torch.no_grad():
         #   x1, x2 = siamese_2(real_B,fake_A)
            SSIM_A2B = 1 -ssim_loss(real_B, recovered_B).detach()
 
        #if SSIM_A2B > .1:
         #   label = 0
        #else:
         #   label = 1 
        
        #same_loss_2 = 2* loss_2(x1, x2,1 )
    
    
        
       # q = same_loss_2
        
 
    
    
    
    
    
    
        #SSIM Loss
        
        #fake_B = netG_A2B(real_A)
        #pred_fake = netD_B(fake_B)
        #SSIM_B2A = -ssim_loss(fake_B, real_A)
        
        
        
    

        #fake_A = netG_B2A(real_B)
        #pred_fake = netD_A(fake_A)
        #SSIM_A2B = -ssim_loss(fake_A, real_B)
        
        

        # Total loss
        loss_G = (loss_identity_A + loss_identity_B + loss_GAN_A2B +
                  loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB + SSIM_B2A + SSIM_A2B)
        
        
        #loss_G = (  loss_GAN_A2B +
         #        loss_identity_A + loss_identity_B+ 
          #         loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB)
        #same_loss_1 + same_loss_2)
        
        
        
        loss_G = loss_G.float()
        
        
        #print(loss_G)
        loss_G.backward()
        
        optimizer_G.step()
        
        
    #           with torch.no_grad(): 
    
        #1st siamese network
        
        #optimizer_1.zero_grad()
        #x1, x2 = siamese_1(recovered_A.detach(), real_A)
        #SSIM_B2A = 1 -ssim_loss(recovered_A, real_A).detach()
        
        
        #label = 0
 
        
            
        #if SSIM_B2A > .1:
         #   label = 0
        #else:
         #   label = 1 
        
        #same_loss_1 = 1* loss_1(x1, x2,label )
        
        
        
        #same_loss =  loss(x1, x2,label )
        #same_loss_1.backward()
        #optimizer_1.step()
        
        
    #            with torch.no_grad():
    
        #2nd siamese network
        
        #optimizer_2.zero_grad()
        #x1, x2 = siamese_2(recovered_B.detach(), real_B)
        #SSIM_A2B = 1 -ssim_loss(recovered_B, real_B).detach()
 
        #if SSIM_A2B > .1:
          #  label = 0
       # else:
            #label = 1 
        
       # same_loss_2 = 1* loss_2(x1, x2,label )
    
        
        
        
        #same_loss =  loss(x1, x2,label )
        #same_loss_2.backward()
       # optimizer_2.step()
        
        
        

        
        
        
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # Fake loss
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_A = ((loss_D_real + loss_D_fake)*0.5)
        loss_D_A.backward()

        optimizer_D_A.step()
        ###################################

        ###### Discriminator B ######
        optimizer_D_B.zero_grad()

        
        
        
        

        
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)
        
        # Fake loss
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_B.step()
        
        
        
        #siamese update
        
        

        
        
        

        
        
        
        

        
                
 
        
        
        
        
                
        if ( epoch % 1)==0 and  (i%400)== 0:
            print("epoch: %d loss_D_A: %f   loss_D_B: %f loss_G: %f ,SSIM_1 :%f  SSIM_2: %f \n"
                  %(epoch, loss_D_A, loss_D_B, loss_G, SSIM_B2A,SSIM_A2B) )
            print(epoch)
            from matplotlib.pyplot import figure
            figure(num=1, figsize=(6, 6))
            print('from game image \n')
            image = fake_B[0].cpu()
            image = (image* .5 + .5)
                    #plt.imshow(image)
            plt.imshow(image.permute(1, 2, 0)  )
            plt.show()
            
            print('from real image')
            figure(num=1, figsize=(6, 6))
            image_2 = fake_A[0].cpu()
                    #plt.imshow(image)
            image_2 = (image_2 *.5 + 0.5  )
            plt.imshow(image_2.permute(1, 2, 0)  )        
            
            
            plt.show()
        
        
        
        
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()
 
    #scheduler_1.step()
    #scheduler_2.step()


# Translating Synthetic Image to Real Image

In [ ]:
epochs = 1

for epoch in range(epochs):
    
    for i, data in enumerate(train_loader):        
        
        real_A = Variable(data['image'].cuda())
        real_B = Variable(data['image_2'].cuda())
        
        
        x1, x2 = siamese_1(real_A,netG_A2B(real_A))
        test_1 = loss(x1, x2, 1)
        
        ssim =  1 - ssim_loss(real_A, (netG_A2B(real_A)))
        print("test_1 : %f, ssim : %f"%(test_1, ssim))
        
        fake_B = 0.5*(netG_A2B(real_A).data + 1.0)
        fake_A = 0.5*(netG_B2A(real_B).data + 1.0)

    # Save image files
        from matplotlib.pyplot import figure
        figure(num=3,figsize = (8,8))
        #plt.imshow(((data['image'][0].cpu()).permute(1, 2, 0)+1)*.5)
        #plt.imsave( '/home/abhijit/Documents/game_output/'+str(i)+ 'a.png', ((data['image'][0].cpu()).permute(1, 2, 0)+1)*.5 )    

        from matplotlib.pyplot import figure
        figure(num=1, figsize=(8, 8))
        #plt.imshow((fake_B[0].cpu()).permute(1, 2, 0))
        plt.imsave( '/home/abhijit/Documents/finalgamee/cycleandssim/'+data['img_2_name'][0]+ '.png' ,(fake_B[0].cpu()).permute(1, 2, 0))
        
        
        figure(num=2, figsize=(8, 8))
        #plt.imshow((fake_A[0].cpu()).permute(1, 2, 0))
        #plt.imsave( '/home/abhijit/Documents/boleizhou_output/siamese_identity/'+data['img_2_name'][0]+'.png' ,(fake_A[0].cpu()).permute(1, 2, 0))
 

        figure(num=4, figsize = (8,8))
        #plt.imshow(((data['image_2'][0].cpu()).permute(1, 2, 0)+1)*.5)
        #plt.imsave( '/home/abhijit/Documents/output2/from real/'+str(i) +'a.png', ((data['image_2'][0].cpu()).permute(1, 2, 0)+1)*.5)

                
        if ( epoch % 5)==0 and  (i%400)== 0:
            from matplotlib.pyplot import figure
            figure(num=1, figsize=(10, 10))
            print('from game image \n')
            image = netG_B2A(fake_B).detach()
            image = image[0].cpu()
            image = (image* .5+.5)
                    #plt.imshow(image)
            plt.imshow(image.permute(1, 2, 0)  )
            plt.show()
            
            #print('from real image')
            #figure(num=1, figsize=(10, 10))
            #image_2 = fake_A[0].cpu()
                    #plt.imshow(image)
            #image_2 = (image_2 *.5 +.5  )
            #plt.imshow(image_2.permute(1, 2, 0)  )        
            
            
            #plt.show()